In [1]:
#Airbnb Listing ID and Link Scraper
#This program scrapes the links and unique listing_ids of the listings inside the search results for Airbnb Queries
#It contains a while loop that compares the old url to the new url of the next page of the search results
#If the old url is the same as new url, that means that is the end of the search results (Max is usually 17 pages)
#The links and IDs are then saved in a dictionary and saved in csv files based on the queries being used

In [124]:
#import all libraries

from __future__ import print_function, division
from bs4 import BeautifulSoup #importing soup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
#importing libraries
import re
import requests
import time
import random
import pandas as pd
requests.__path__

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [205]:
#setup queries for different locations
downtown = 'https://www.airbnb.com/s/Lower-Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&search_type=autocomplete_click&checkin=2019-12-25&checkout=2019-12-26&adults=1&place_id=ChIJi4MZVIhZwokRPNvWbi-c0wI&s_tag=S2nyeCsb'
midtown = 'https://www.airbnb.com/s/Midtown--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&place_id=ChIJqXwSpAFZwokR28_WgZDMzb4&search_type=autocomplete_click&checkin=2019-12-25&checkout=2019-12-26&adults=1&s_tag=y_k8JEnt'
centralpark = 'https://www.airbnb.com/s/Central-Park--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&place_id=ChIJ4zGFAZpYwokRGUGph3Mf37k&search_type=autocomplete_click&checkin=2019-12-25&checkout=2019-12-26&adults=1&s_tag=0uae9TOm'
uptown= 'https://www.airbnb.com/s/Upper-Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&place_id=ChIJY50xHxP2wokR7WhhDNRmgRU&search_type=autocomplete_click&checkin=2019-12-25&checkout=2019-12-26&adults=1&s_tag=f-00FCHO'

downtown_shared = 'https://www.airbnb.com/s/Downtown--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&search_type=unknown&zoom=14&search_by_map=true&sw_lat=40.70267570229368&sw_lng=-74.03261788195204&ne_lat=40.755425412760644&ne_lng=-73.96180756395887&checkin=2019-12-25&checkout=2019-12-26&adults=1&room_types%5B%5D=Shared%20room&place_id=ChIJi4MZVIhZwokRPNvWbi-c0wI&s_tag=JE0WGUVW'
midtown_shared = 'https://www.airbnb.com/s/Midtown--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&search_type=unknown&zoom=15&search_by_map=true&sw_lat=40.74177156077865&sw_lng=-74.00153543426427&ne_lat=40.76813614825684&ne_lng=-73.96613027526769&checkin=2019-12-25&checkout=2019-12-26&adults=1&room_types%5B%5D=Shared%20room&place_id=ChIJqXwSpAFZwokR28_WgZDMzb4&s_tag=P7xWps2U'
centralpark_shared = 'https://www.airbnb.com/s/Central-Park--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&place_id=ChIJ4zGFAZpYwokRGUGph3Mf37k&search_type=autocomplete_click&checkin=2019-12-25&checkout=2019-12-26&adults=1&room_types%5B%5D=Shared%20room&s_tag=9MvPjDYp'
uptown_shared = 'https://www.airbnb.com/s/uptown--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&search_type=search_query&checkin=2019-12-25&checkout=2019-12-26&adults=1&room_types%5B%5D=Shared%20room&place_id=ChIJY50xHxP2wokR7WhhDNRmgRU&s_tag=AVDnunhc'
#define driver
driver = webdriver.Chrome(chromedriver)

#define manhattan sample
manhattan = [uptown, downtown_shared, midtown_shared, centralpark_shared, uptown_shared]

#driver get link
id_list = []
link_list = []

#url for previous page
old_url = ""

#counter for page nunmber

counter = 1



In [206]:
#loop through getting links and ids
driver.get(manhattan[4])

#restart = "https://www.airbnb.com/s/Upper-Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&place_id=ChIJY50xHxP2wokR7WhhDNRmgRU&search_type=pagination&s_tag=f-00FCHO&section_offset=7&items_offset=216&checkin=2019-12-25&checkout=2019-12-26&adults=1"
#driver.get(restart)

In [207]:
while old_url != driver.current_url:
    #### WAIT FOR PAGE LOADS
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "_r4n1gzb")))
    except:
        print("Page Taking Longer to Load")
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "_1dss1omb")))
    #### PAGE LOADED
    
    #soupify or reupdate and soupify html in soup AFTER PAGE LOADS
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #Get unique listing ids
    listings = soup.findAll(class_ ='_qlq27g') #this is the class that ids are located in
    for div in listings:
        id_list.append(div.find('a')['target'])
        
    #Get links for next scraper
    links = soup.findAll(class_ ='_qlq27g') #find all the links
    for div in links:
        link_list.append('https://www.airbnb.com' + div.find('a')['href'])
        
    
    #Print page number scraped
    
    print(counter)
    
    counter = counter + 1
    
    #Move to Next Page in Search Results also adding timedelay so that we don't get blocked
    
    time.sleep(15)
    
    #time.sleep(7+2*random.random()) #random sleep to mimic a human to be safe make it 9 seconds at least
    try:    
        next_url = soup.find('li', {'class': '_r4n1gzb'}).a['href']#find next_url 
    
    except:
        
        print('Exception Occurred, trying again')
   
        continue #continues next loop not next line
   
    
    #set old_url to current_url for comparisson in for loop
    old_url = driver.current_url
    
    driver.get("https://www.airbnb.com"+next_url) #move to new page
    
    

1
2


In [208]:
len(link_list)

31

In [209]:
len(id_list)

31

In [210]:
driver.close()

In [211]:
#clean id_list
new_list =[]
for listing in id_list:
    listing = listing.replace("listing_", '')
    
    new_list.append(listing)
    
    print (new_list)
    
id_list = new_list

['31103138']
['31103138', '31267520']
['31103138', '31267520', '34791645']
['31103138', '31267520', '34791645', '27045165']
['31103138', '31267520', '34791645', '27045165', '36077664']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404', '6155877']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404', '6155877', '36286131']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404', '6155877', '36286131', '34124248']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404', '6155877', '36286131', '34124248', '23461056']
['31103138', '31267520', '34791645', '27045165', '36077664', '10126128', '36075404', '6155877', '36286131', '34124248', '23461056', '14655004']
['31103138', '31267520', '34791645', '27045165', '3607766

In [212]:
id_list

['31103138',
 '31267520',
 '34791645',
 '27045165',
 '36077664',
 '10126128',
 '36075404',
 '6155877',
 '36286131',
 '34124248',
 '23461056',
 '14655004',
 '11627123',
 '15174230',
 '28860221',
 '34624240',
 '33987446',
 '34665824',
 '34665824',
 '34718463',
 '7711899',
 '5046894',
 '34718633',
 '13001082',
 '34126672',
 '34782691',
 '3979714',
 '851130',
 '3120957',
 '3550879',
 '6991749']

In [215]:
#Create a dataframe from lists generated

df = pd.DataFrame(list(zip(id_list, link_list)), columns =['ID','Link' ]) 

df

,ID,Link
0,31103138,https://www.airbnb.com/rooms/31103138?location...
1,31267520,https://www.airbnb.com/rooms/31267520?location...
2,34791645,https://www.airbnb.com/rooms/34791645?location...
3,27045165,https://www.airbnb.com/rooms/27045165?location...
4,36077664,https://www.airbnb.com/rooms/36077664?location...
5,10126128,https://www.airbnb.com/rooms/10126128?location...
6,36075404,https://www.airbnb.com/rooms/36075404?location...
7,6155877,https://www.airbnb.com/rooms/6155877?location=...
8,36286131,https://www.airbnb.com/rooms/36286131?location...
9,34124248,https://www.airbnb.com/rooms/34124248?location...


In [216]:
manhattan_labels = ['downtown','midtown','uptown','centralpark']

df = df.assign(Neighborhood ='uptown')

df

,ID,Link,Neighborhood
0,31103138,https://www.airbnb.com/rooms/31103138?location...,uptown
1,31267520,https://www.airbnb.com/rooms/31267520?location...,uptown
2,34791645,https://www.airbnb.com/rooms/34791645?location...,uptown
3,27045165,https://www.airbnb.com/rooms/27045165?location...,uptown
4,36077664,https://www.airbnb.com/rooms/36077664?location...,uptown
5,10126128,https://www.airbnb.com/rooms/10126128?location...,uptown
6,36075404,https://www.airbnb.com/rooms/36075404?location...,uptown
7,6155877,https://www.airbnb.com/rooms/6155877?location=...,uptown
8,36286131,https://www.airbnb.com/rooms/36286131?location...,uptown
9,34124248,https://www.airbnb.com/rooms/34124248?location...,uptown


In [217]:
df['ID'].nunique()

30

In [218]:
df.to_csv('uptown_shared.csv', index = False)